In [1]:
# Import Dependencies

import json
import numpy as np
import pandas as pd
import matplotlib as plt
import requests
import time
from config import api_key


In [ ]:
#total_pop = "B01001_001E"

## Age Group Codes
#Males age 18 to 19 = B01001_007E
#Males age 20 = B01001_008E
#Males age 21 = B01001_009E
#Males age 22 to 24 = B01001_010E
#Males age 25 to 29 = B01001_011E
#Males age 30 to 34 = B01001_012E
#Males age 35 to 39 = B01001_013E

#Females age 18 to 19 = B01001_031E
#Females age 20 = B01001_032E
#Females age 21 = B01001_033E
#Females age 22 to 24 = B01001_034E
#Females age 25 to 29 = B01001_035E
#Females age 30 to 34 = B01001_036E
#Females age 35 to 39 = B01001_037E

## Ethnicity Group Codes
#White = B02001_002E
#Black = B02001_003E
#Latino = B03001_001E
#Native Indian = B02001_004E
#Asian = B02001_005E
#Pacific Islander = B02001_006E
#Other = B02001_007E

## Income Level
#income_pop = B25095_001E
#Less than $10,000 = B25095_002E
#$10,000 to $19,999 = B25095_011E
#$20,000 to $34,999 = B25095_020E
#$35,000 to $49,999 = B25095_029E
#$50,000 to $74,999 = B25095_038E
#$75,000 to $99,999 = B25095_047E
#$100,000 to $149,999 = B25095_056E
#$150,000 or more = B25095_065E

## Education Level
#edu_pop = B23006_001E
#Less than high school graduate = B23006_002E
#High school graduate = B23006_009E
#Some college or associates = B23006_016E
#Bachelor's or higher = B23006_023E


In [2]:
# Build query URL for exploring Census API

# Base URL
url = "https://api.census.gov/data/2016/acs/acs5?get=NAME"

# API Key call, note this should always be at the end of the URL
key = "&key=" + api_key

# Call for states, note that the State's designated number should follow the colon
state = "&in=state:"

# Call for individual counties, note that you must make the county call with the state call, it cannot be called
# by itself. Also, the county number should follow the colon
county = "&for=county:"

# Places API Call, which allows for specific cities to be referred to. Requires the "state" call to be made as well,
# otherwise it might return an error.

city = "&for=place:"

# Metropolitan API call

metro = "&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:"

# ZIP code ZCTA API call

zip_code = "&for=zip%20code%20tabulation%20area:"

# Basic query url structure should look like this (Note that the "*" is used to call all values, so all states
# and cities within that state)
query_url = url + city + "04000" + state + "13" + key

# Json request
request = requests.get(query_url).json()

# Display result of request
request


In [3]:
# URLs with codes built in to make appropriate calls

# Age URL
age_url = "https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,"\
        +"B01001_011E,B01001_012E,B01001_013E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_037E"

# Ethnicity URL
ethnicity_url = "https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E,B02001_002E,B02001_003E,B03001_001E,B02001_004E,"\
        +"B02001_005E,B02001_006E,B02001_007E"

# Income URL
income_url = "https://api.census.gov/data/2016/acs/acs5?get=NAME,B25095_001E,B25095_002E,B25095_011E,B25095_020E,"\
        +"B25095_029E,B25095_038E,B25095_047E,B25095_056E,B25095_065E"

# Education URL
edu_url = "https://api.census.gov/data/2016/acs/acs5?get=NAME,B23006_001E,B23006_002E,B23006_009E,B23006_016E,B23006_023E"

# List of URLs for iteration in API

url_list = [age_url,ethnicity_url,income_url,edu_url]


In [4]:
# Dictionary with state names as keys and the values being the corresponding code in the Census API
state_dict = {"Georgia":"13","Washington":"53","Florida":"12","New York":"36","California":"06","New Jersey":"34",\
               "Oregon":"41","Minnesota":"27","Kansas":"20","Massachusetts":"25","Utah":"49","Washington D.C.":"11",\
               "Texas":"48","Illinois":"17","Pennsylvania":"42","Ohio":"39","Colorado":"08"}

# Dictionary with city names as keys and the values being the corresponding code in the Census API
city_dict = {"Atlanta":"04000", "Seattle":"63000", "Orlando":"53000", "New York":"51000", "Carson":"11530",\
              "Harrison":"51000", "Portland":"59000", "Minneapolis":"43000", "San Jose":"68000", "Kansas City":"36000",\
              "Foxborough":"07000", "Sandy":"67440", "Washington D.C.":"50000", "Houston":"35000",\
              "Bridgeview":"14000", "Chester":"60000", "Columbus":"18000", "Commerce City":"20000", "Frisco":"27684"}

# List containing each state code in corresponding order for use in API call iteration
state_codes = ["13","53","12","36","06","34","41","27","06","20","25","49","11","48","17","42","39","08","48"]

# List containing each city code in corresponding order for use in API call iteration
city_codes = ["04000","63000","53000","51000","11530","51000","59000","43000","68000","36000","07000","67440","50000",\
             "35000","14000","60000","18000","20000","27684"]


In [5]:
# API request iteration format (NOTE: running this cell takes time due to the time.sleep! Only run
# if you want the data itself! Running this will take at least a minute and forty seconds before it finishes)

# Initialize lists to store data retrieved from the API

age_list = []
ethnicity_list = []
income_list = []
edu_list = []

counter = 1

for num in range(19):
    
    for url in url_list:
    
        # Query URL construction that pulls from the city & state codes lists
        query_url = url + city + city_codes[num] + state + state_codes[num] + key
    
        request = requests.get(query_url).json()
        
        if url == age_url:
            age_list.append(request[1])
        elif url == ethnicity_url:
            ethnicity_list.append(request[1])
        elif url == income_url:
            income_list.append(request[1])
        elif url == edu_url:
            edu_list.append(request[1])
            
        time.sleep(1)
    
    print(f"#{counter} has completed all calls")
    
    counter += 1


#1 has completed all calls
#2 has completed all calls
#3 has completed all calls
#4 has completed all calls
#5 has completed all calls
#6 has completed all calls
#7 has completed all calls
#8 has completed all calls
#9 has completed all calls
#10 has completed all calls
#11 has completed all calls
#12 has completed all calls
#13 has completed all calls
#14 has completed all calls
#15 has completed all calls
#16 has completed all calls
#17 has completed all calls
#18 has completed all calls
#19 has completed all calls
